# Import required packages

In [15]:
import os
import sys
import pandas as pd
import requests
import json
from datetime import datetime, date
from tqdm import tqdm
from dotenv import load_dotenv


# Credentials & Token

Save your cdse log in credentials to the .env file as:
CDSE_email = "your_email"
CDSE_password = "your_password"

In [14]:
load_dotenv()
username=os.getenv("CDSE_email")
password=os.getenv("CDSE_password")

# check that the credentials are set correctly 
if not password or not username:
    print("Please provide your Sentinel Hub credentials in the .env file.")
    exit(1)

In [16]:
def get_access_token(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username":username,
        "password":password,
        "grant_type": "password",
        }
    try:
        r = requests.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Access token creation failed. Reponse from the server was: {r.json()}"
            )
    print("Access token created successfully!")
    return r.json()["access_token"]

In [17]:
# Get access token
token = get_access_token(username, password)

Access token created successfully!


# Set Search Criteria

In [18]:
data_collection = "SENTINEL-2"
bbox = [12.45, 44.825, 12.7, 45.055]    # Po River
#bbox = [9.6, 42.95, 9.9, 43.155]       # North East Corsica
#bbox = [16.5, 38.35, 16.755, 38.555]   # South East Calabria
start_date =  "2019-07-26"
end_date = "2019-07-31"
output_dir = "../data/SAFE"

In [ ]:
https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq 'CCM' and OData.CSC.Intersects(area=geography'SRID=4326;POLYGON((12.655118166047592 47.44667197521409,21.39065656328509 48.347694733853245,28.334291357162826 41.877123516783655,17.47086198383573 40.35854475076158,12.655118166047592 47.44667197521409))')&$top=20

In [24]:
def get_url(data_collection, bbox, start_date, end_date, token):
  cloud_cover = 20
  headers = {"Authorization": f"Bearer {token}"}
  response = requests.get(
    f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq '{data_collection}' "  
    f"and OData.CSC.Intersects(area=geography'SRID=4326;{bbox}) "  
    f"and ContentDate/Start ge {start_date}T00:00:00.000Z and ContentDate/Start le {end_date}T00:00:00.000Z "  
    f"and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/OData.CSC.StringAttribute/Value eq 'productType' and att/Value eq 'S2MSI1C')",
    headers=headers)
  response.raise_for_status()
  return response.json()

In [13]:
url = (
f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?"
f"$filter=Collection/Name eq'{data_collection}' and "
f"OData.CSC.Intersects(area=geography'SRID=4326;"
f"POLYGON(({str(bbox[0]) + " " + str(bbox[1]) + "," + str(bbox[2]) + " " + str(bbox[1]) + "," + str(bbox[2]) + " " + str(bbox[3]) + ","  + str(bbox[0]) + " " + str(bbox[3]) + "," + str(bbox[0]) + " " + str(bbox[1])}'))') and "
f"ContentDate/Start gt {start_date}T00:00:00.000Z and "
f"ContentDate/Start lt {end_date}T00:00:00.000Z)"
)
print(url)

https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq'SENTINEL-2' and OData.CSC.Intersects(area=geography'SRID=4326;POLYGON((12.45 44.825,12.7 44.825,12.7 45.055,12.45 45.055,12.45 44.825'))') and ContentDate/Start gt 2019-07-26T00:00:00.000Z and ContentDate/Start lt 2019-07-31T00:00:00.000Z)


In [21]:
def download_product(token, product_id, output_path):
    url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"
    headers = {"Authorization": f"Bearer {token}"}
    session = requests.Session()
    session.headers.update(headers)

    response = session.get(url, allow_redirects=False)
    while response.status_code in (301, 302, 303, 307):
        url = response.headers["Location"]
        response = session.get(url, allow_redirects=False)

    final_response = session.get(url, stream=True)
    final_response.raise_for_status()

    with open(output_path, "wb") as f:
        for chunk in final_response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"Downloaded to {output_path}")

# run download

In [25]:
result_2 = get_url(data_collection, bbox, start_date, end_date, token)
print("Full API response:", result_2)
if not result_2["value"]:
    print("No Sentinel-2 L1C products found for the specified criteria.")
    exit()

product = result_2["value"][0]
product_id = product["Id"]
product_name = product["Name"]
output_zip = f"{product_name}.zip"

# Download the product
download_product(token, product_id, output_zip)


HTTPError: 403 Client Error: Forbidden for url: https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name%20eq%20'SENTINEL-2'%20and%20OData.CSC.Intersects(area=geography'SRID=4326;%5B12.45,%2044.825,%2012.7,%2045.055%5D)%20and%20ContentDate/Start%20ge%202019-07-26T00:00:00.000Z%20and%20ContentDate/Start%20le%202019-07-31T00:00:00.000Z%20and%20Attributes/OData.CSC.StringAttribute/any(att:att/Name%20eq%20'productType'%20and%20att/OData.CSC.StringAttribute/Value%20eq%20'productType'%20and%20att/Value%20eq%20'S2MSI1C')

In [ ]:
# alt request url retrieval:

def get_https_request(satellite, contains, start_date, end_date, token):   #  contains2 
    # catalogue specification
    base_prefix = "https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter="
    # search criteria
    collection = "Collection/Name eq '" + satellite + "' and contains(Name,'" + contains + "')" # and contains(Name,'" + contains2 + "'
    roi_coordinates = str(bbox[0]) + " " + str(bbox[1]) + "," + str(bbox[2]) + " " + str(bbox[1]) + "," + str(bbox[2]) + " " + str(bbox[3]) + "," + str(bbox[0]) + " " + str(bbox[3]) + "," + str(bbox[0]) + " " + str(bbox[1])
    geographic_criteria = "OData.CSC.Intersects(area=geography'SRID=4326;POLYGON((" + roi_coordinates + "))') "
    content_date = (
            "ContentDate/Start gt " + start_date + "T00:00:00.000Z and " +
            "ContentDate/Start lt " + end_date + "T00:00:00.000Z"
    )
    # attributes = "Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/Value eq 'S2MSI1C')"
    
    # Top option specifies the maximum number of items returned from a query.
    # The default value is set to 20.
    # The acceptable arguments for this option: Integer <0,1000>
    top_option = "&$top=1000"
    
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    
    https_request = (
            base_prefix + collection + " and " + geographic_criteria + " and " + content_date + top_option
            )
    print("Query URL:", https_request)
    return https_request

#then move and apply requests.get() and raise_for_status() outside the function/in new function

# Iterate (just notes)

In [6]:
http_rs = [] 
for i in matching_ids:
    
    satellite = "Sentinel-2"
    contains = "S2MSI1C"
    contains2 = i['id'][11:26]
    http_request = get_https_request(satellite, contains, start_date, end_date)
    http_rs.append(http_request)
    return http_rs

SyntaxError: 'return' outside function (1571520283.py, line 9)

In [ ]:
# Load the CSV file
matched_products_df = pd.read_csv("../matched_s2_products.csv")
unique_products_df = matched_products_df.drop_duplicates(subset="product_id")

# Create the dictionary
filter_products_dict = {
    "Name" : [{product_id} for product_id in unique_products_df["product_id"]]
}

len(filter_products_dict["Name"])

3977

can also search by product name (see [docs](https://documentation.dataspace.copernicus.eu/APIs/OData.html)), but two issues:
- matched_s2_products has 3977 tiles, s2_product_unique has 4472 (but outdated N0XXX in product name),
i.e. needs to be limited spatially/temporally at minimum
- need to change functions to deal with iterating through dict values 

possible accepted url options:

In [ ]:
filter_products_dict.keys()
filter_products_dict.values()

In [ ]:

satellite = "Sentinel-2"
contains = "L1C"
contains2 = "T32TQQ"    # option to be replaced with unique tiles or strtime iterating through 
                        # litter rows csv within matching_ids, i.e. iterated over